<a href="https://colab.research.google.com/github/shinchacoffee/2_circles_eparation_simple/blob/main/pytorch_tutorial_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
training_data = datasets.FashionMNIST(root='data', 
                                      train=True, 
                                      download=True,
                                      transform=ToTensor()
                                      )

test_data = datasets.FashionMNIST(root='data',
                                  train=False,
                                  download=True,
                                  transform=ToTensor()
                                  )

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_data_loader = DataLoader(training_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_data_loader:
  print(f"Shape of features (X) [N, C, H, W]: {X.shape}")
  print(f"Shape of labels (y): {y.shape} {y.dtype}")
  break


Shape of features (X) [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of labels (y): torch.Size([64]) torch.int64


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

using cuda device


In [26]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(512),
        nn.Linear(512,512),
        nn.ReLU(512),
        nn.Linear(512,10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, lossfn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = lossfn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, lossfn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += lossfn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"test error: \n accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")

In [28]:
#model.load_state_dict(torch.load("model.pth"))
epochs = 1
for t in range(epochs):
  print(f"epoch {t+1}\n------------------------------")
  model.to(device)
  train(train_data_loader, model, loss_fn, optimizer)
  test(test_data_loader, model, loss_fn)
print("Done! lul :x")

epoch 1
------------------------------
loss: 0.218189 [    0/60000]
loss: 0.331583 [ 6400/60000]
loss: 0.215969 [12800/60000]
loss: 0.332476 [19200/60000]
loss: 0.276272 [25600/60000]
loss: 0.300061 [32000/60000]
loss: 0.316481 [38400/60000]
loss: 0.450796 [44800/60000]
loss: 0.416131 [51200/60000]
loss: 0.340159 [57600/60000]
test error: 
 accuracy: 87.0%, avg loss: 0.368096 

Done! lul :x


In [23]:
torch.save(model.state_dict(), "model.pth")
print("saved pytorch model state to model.pth")

saved pytorch model state to model.pth


In [25]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model2 = NeuralNetwork()
model2.load_state_dict(torch.load("model.pth"))

model.to("cpu")
model.eval()
x, y = test_data[1][0], test_data[1][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
